In [28]:
import sys
!{sys.executable} -m pip install xgboost
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt  
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix
train = pd.read_excel (r'C:\Users\DELL\Downloads\Train_dataset.xlsx')
test = pd.read_excel (r'C:\Users\DELL\Downloads\Test_dataset.xlsx')
print (train)
print (test)

                   City           State    Type  Population [2011]  \
0                Mumbai     Maharashtra     M.C         12442373.0   
1                 Delhi           Delhi     M.C         11007835.0   
2             Bangalore       Karnataka    MPUA          8436675.0   
3             Hyderabad       Telangana    MPUA          6809970.0   
4             Ahmedabad         Gujarat    MPUA          5570585.0   
5               Chennai      Tamil Nadu    MPUA          4681087.0   
6               Chennai      Tamil nadu       T          4646732.0   
7               Kolkata     West Bengal    MPUA          4486679.0   
8                 Surat         Gujarat    MPUA          4467797.0   
9                  Pune     Maharashtra    MPUA          3124458.0   
10               Jaipur       Rajasthan    MPUA          3046163.0   
11              Lucknow   Uttar Pradesh    MPUA          2817105.0   
12               Indore  Madhya Pradesh       M          2800000.0   
13               Kan

In [29]:
swm = pd.get_dummies(train['SWM'], drop_first=True)
swm_test = pd.get_dummies(test['SWM'], drop_first=True)
train = pd.concat([train, swm], axis = 1)
test = pd.concat([test, swm_test], axis = 1)

In [30]:
train.drop(['City', 'Popuation [2001]', 'State', 'SWM', 'Sex Ratio', 'Median Age', 'Avg Temp', 'Water Purity', 'H Index'], axis = 1, inplace= True)
test.drop(['City','Popuation [2001]', 'State', 'SWM', 'Sex Ratio', 'Median Age', 'Avg Temp', 'Water Purity', 'H Index'], axis = 1, inplace= True)

In [31]:

amp_object = train.select_dtypes(include = np.object)
amp_int = train.select_dtypes(exclude=np.object)
amp_object1 = test.select_dtypes(include = np.object)
amp_int1 = test.select_dtypes(exclude=np.object)

In [32]:
for clm in amp_object.columns:
    train[clm] = train[clm].fillna(value = train[clm].mode())
    #print(train[clm].mode())
    
for clm in amp_int.columns:
    train[clm] = train[clm].fillna(value = train[clm].mean())



for clm in amp_object1.columns:
    test[clm] = test[clm].fillna(value = test[clm].mode())
    #print(test[clm].mode())
    
for clm in amp_int1.columns:
    test[clm] = test[clm].fillna(value = test[clm].mean())

In [33]:
d = {'Type': ['C-1T','M','M.Cl','MPUA','M.B','UA']}
typ = np.array(train['Type'])
for i in range(len(train)):
    if typ[i] not in d['Type']:
        typ[i]='other_type'

train['Type'] = typ
obj_clm = train.select_dtypes(include = np.object)
for c in obj_clm.columns:
    k = train[c].value_counts()
    print(c, '***'*2,'\n' ,k)
    
lst1 = []


typ1 = np.array(test['Type'])
for i in range(len(test)):
    lst1.append(0)
    if typ1[i] not in d['Type']:
        typ1[i]='other_type'

test['Type'] = typ1
obj_clm1 = test.select_dtypes(include = np.object)
for c in obj_clm1.columns:
    k = test[c].value_counts()
    print(c, '***'*2,'\n' ,k)
test['Type_MPUA'] = np.array(lst1)
test['Type_M'] = np.array(lst1)

Type ****** 
 C-1T          269
M             236
other_type    123
M.Cl           59
MPUA           44
M.B            28
UA             28
Name: Type, dtype: int64
Type ****** 
 other_type    297
M             119
M.Cl           61
M.B            16
UA              8
Name: Type, dtype: int64


In [34]:
test.head()


,Type,Population [2011],Toilets Avl,Female Population,# of hospitals,Foreign Visitors,LOW,MEDIUM,Type_MPUA,Type_M
0,other_type,36774.0,94.0,34237.0,17.0,2769.0,0,1,0,0
1,other_type,36754.0,62.0,34328.0,13.0,636502.0,0,0,0,0
2,other_type,36732.0,63.0,32434.0,17.0,242367.0,0,0,0,0
3,M.Cl,36706.0,60.0,32558.0,11.0,4408916.0,0,0,0,0
4,other_type,36669.0,92.0,32159.0,23.0,3104060.0,1,0,0,0


In [35]:
train.drop(obj_clm.columns,axis=1,inplace =True)
obj_clm = pd.get_dummies(obj_clm,drop_first = True)


test.drop(obj_clm1.columns,axis=1,inplace =True)
obj_clm1 = pd.get_dummies(obj_clm1,drop_first = True)

In [36]:
train = pd.concat([train,obj_clm],axis =1)
train['total_toilets'] = train['# of hospitals'] * train['Toilets Avl']

test = pd.concat([test, obj_clm1],axis =1)
test['total_toilets'] = test['# of hospitals'] * test['Toilets Avl']

In [37]:
train.drop(['# of hospitals','Toilets Avl'],inplace = True,axis=1)
test.drop(['# of hospitals','Toilets Avl'],inplace = True,axis=1)

In [38]:
train.drop(['Female Population'] , axis=1, inplace=True)
test.drop(['Female Population'] , axis=1, inplace=True)

In [39]:
test.head()

,Population [2011],Foreign Visitors,LOW,MEDIUM,Type_MPUA,Type_M,Type_M.B,Type_M.Cl,Type_UA,Type_other_type,total_toilets
0,36774.0,2769.0,0,1,0,0,0,0,0,1,1598.0
1,36754.0,636502.0,0,0,0,0,0,0,0,1,806.0
2,36732.0,242367.0,0,0,0,0,0,0,0,1,1071.0
3,36706.0,4408916.0,0,0,0,0,0,1,0,0,660.0
4,36669.0,3104060.0,1,0,0,0,0,0,0,1,2116.0


In [58]:
x = train.drop('Covid Cases' , axis=1)
y = train['Covid Cases']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
import re
import pandas as pd
import numpy as np



In [61]:
from xgboost.sklearn import XGBRegressor
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

# test input data with string, int, and symbol-included columns 
df = pd.DataFrame({'0': np.random.randint(0, 2, size=100),
                   '[test1]': np.random.uniform(0, 1, size=100),
                   'test2': np.random.uniform(0, 1, size=100),
                  3: np.random.uniform(0, 1, size=100)})

df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df.columns.values]

target = df.iloc[:, 0]
predictors = df.iloc[:, 1:]
xgb0 = XGBRegressor(objective= 'reg:linear')
xgb0.fit(predictors, target)
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score as acc
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

linear = LinearRegression()
sv = SVR()
rf = RandomForestRegressor(random_state=72)
xg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.01,
                max_depth = 5, alpha = 10, n_estimators = 10)

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
params1={
 "max_depth"        :  (3, 4, 5, 6, 8, 10, 12, 15),
 "n_estimators"     : (100,300,500,700,800,1000),
 "max_features"     : (4,6,8,10)
}
random_search0=RandomizedSearchCV(rf,param_distributions=params1,n_iter=5,n_jobs=-1,cv=5,verbose=3)
random_search0.fit(x_train, y_train)



linear.fit(x_train,y_train)
sv.fit(x_train,y_train)
rf.fit(x_train,y_train)
xg.fit(x_train,y_train)
l = (linear,sv,rf, random_search0, xg)
for model in l:
    print(str(model))
    y = model.predict(x_test)
    print("*"*5,r2_score(y, y_test))
    sns.kdeplot(y-y_test)
    plt.show()

[20:21:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[20:21:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:    1.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    1.4s finished


ValueError: feature_names must be string, and may not contain [, ] or <

In [49]:
rf.score(x_train, y_train)

0.8899538412097252

In [50]:
train.head()

,Population [2011],Foreign Visitors,Covid Cases,LOW,MEDIUM,Type_M,Type_M.B,Type_M.Cl,Type_MPUA,Type_UA,Type_other_type,total_toilets
0,12442373.0,4408916.0,163115,0,1,0,0,0,0,0,1,34821.0
1,11007835.0,2379169.0,80188,0,1,0,0,0,0,0,1,31820.0
2,8436675.0,636502.0,141000,0,0,0,0,0,1,0,0,26076.0
3,6809970.0,126078.0,55123,0,1,0,0,0,1,0,0,23870.0
4,5570585.0,284973.0,33204,1,0,0,0,0,1,0,0,16571.0


In [51]:
test.head()

,Population [2011],Foreign Visitors,LOW,MEDIUM,Type_MPUA,Type_M,Type_M.B,Type_M.Cl,Type_UA,Type_other_type,total_toilets
0,36774.0,2769.0,0,1,0,0,0,0,0,1,1598.0
1,36754.0,636502.0,0,0,0,0,0,0,0,1,806.0
2,36732.0,242367.0,0,0,0,0,0,0,0,1,1071.0
3,36706.0,4408916.0,0,0,0,0,0,1,0,0,660.0
4,36669.0,3104060.0,1,0,0,0,0,0,0,1,2116.0


In [52]:
yr = rf.predict(test)

In [53]:
lst = []
for i in yr:
    lst.append(int(i))
    
df['Result'] = pd.DataFrame(lst)
df.head()

,City,State,Type,Population [2011],Popuation [2001],Sex Ratio,Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Foreign Visitors,Covid Cases,Result
0,Mumbai,Maharashtra,M.C,12442373.0,11978450.0,878.0,23.0,32.0,MEDIUM,219.0,150.0,0.700440,10924403.0,159.0,4408916.0,163115,2808.0
1,Delhi,Delhi,M.C,11007835.0,9879172.0,858.0,27.0,30.0,MEDIUM,215.0,196.0,0.920018,9444722.0,148.0,2379169.0,80188,2363.0
2,Bangalore,Karnataka,MPUA,8436675.0,4301326.0,936.0,28.0,37.0,HIGH,212.0,102.0,0.097085,7896728.0,123.0,636502.0,141000,2639.0
3,Hyderabad,Telangana,MPUA,6809970.0,3637483.0,930.0,23.0,31.0,MEDIUM,217.0,118.0,0.827744,6333272.0,110.0,126078.0,55123,2283.0
4,Ahmedabad,Gujarat,MPUA,5570585.0,3520085.0,852.0,29.0,25.0,LOW,227.0,109.0,0.847941,4746138.0,73.0,284973.0,33204,2329.0


In [54]:
df.to_csv('result.csv')